In [2]:
%load_ext autoreload
import pandas as pd
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import glob
from tqdm.notebook import tqdm
import os

In [3]:
%autoreload 1

In [4]:
path = '../ignored/tvsum50_ver_1_1/ydata-tvsum50-v1_1/data/ydata-tvsum50-anno.tsv'
df = pd.read_csv(path,delimiter='\t', header=None, names=['id','cat','scores'])
df = df.join(df['scores'].str.split(',',expand=True).apply(pd.to_numeric))
df = df.drop(axis=1, columns=['scores'])

In [5]:
import sys
sys.path.append('../src')
%aimport evaluation

In [106]:
files = []
for file_id in glob.glob('../inputs/*'):
    filename = file_id.split('/')[-1].split('.')[0]
    if filename not in files:
        files.append(filename)
order = [ "AwmHb44_ouw", "98MoyGZKHXc", "J0nA4VgnoCo", "gzDbaEs1Rlg", "XzYM3PfTM4w", "HT5vyqe0Xaw", "sTEELN-vY30", "vdmoEJ5YbrQ", "xwqBXPGE9pQ", "akI8YFjEmUw", "i3wAGJaaktw", "Bhxk-O1Y7Ho", "0tmA_C6XwfM", "3eYKfiOEJNs", "xxdtq8mxegs", "WG0MBPpPC6I", "Hl-__g2gn_A", "Yi4Ij2NM7U4", "37rzWOQsNIw", "LRw_obCPUt0", "cjibtmSLxQ4", "b626MiF1ew4", "XkqCExn6_Us", "GsAD1KT1xo8", "PJrm840pAUI", "91IHQYk1IQM", "RBCABdttQmI", "z_6gVvQb2d0", "fWutDQy1nnY", "4wU_LUjG5Ic", "VuWGsYPqAX8", "JKpqYvAdIsw", "xmEERLqJ2kU", "byxOvuiIJV0", "_xMr-HKMfVA", "WxtbjNsCQ8A", "uGu_10sucQo", "EE-bNr36nyA", "Se3oxnaPsz0", "oDXZc0tZe04", "qqR6AEXwxoQ", "EYqVtI9YWJA", "eQu1rNs0an0", "JgHubY5Vw3Y", "iVt07TCkFM0", "E11zDS9XGzg", "NyBmCxDoHJU", "kLxoNp-UchI", "jcoYJXDG9sw", "-esJrBWj2d8"]

files = sorted(files, key={k:v for v,k in enumerate(order)}.get)

In [118]:
all_val_list = []
clip_fps = []
for file_id in tqdm(files):
    frame_scores_for_video = df[df['id']==file_id].drop(axis=1, columns=['id','cat']).values
    frame_scores_for_video_nonan = frame_scores_for_video[~np.isnan(frame_scores_for_video)]
    frame_scores_for_video_nonan = frame_scores_for_video_nonan.reshape((20,-1))
    temp = []
    if file_id in files:
        with open(f'../intermediate/{file_id}_clip_list.json') as f:
            clip_data = json.load(f)

        # Convert clip start time and end time to fps
        fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
        for clip in clip_data:
            h,m,s = clip['clip_start'].split(':')
            secs = int(h)*3600+int(m)*60+float(s)
            clip['clip_start_fps'] = round(secs*fps)
            h,m,s = clip['clip_end'].split(':')
            secs = int(h)*3600+int(m)*60+float(s)
            clip['clip_end_fps'] = round(secs*fps)
            temp.append([clip['clip_start_fps'], clip['clip_end_fps']])
        # Generate list with included and excluded frames. Included frames are 1.
        val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
        try:
            for clip in clip_data:
                for i in range(clip['clip_start_fps'], clip['clip_end_fps']+1):
                    val_list[i] = 1
            val_np = np.array(val_list)
        except:
            pass
        
    elif file_id in order:
        print(file_id)
        val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
        temp.append([])
    all_val_list.append(val_list)
    clip_fps.append(temp)
#     for clip in clip_data:
#         print(clip['clip_start_fps'], clip['clip_end_fps'])

    # Plot the average score graph
    plt.plot(frame_scores_for_video_nonan.mean(axis=0))
    
    # Plot the regions of the selected frames
    for i in range(len(val_list)):
        if val_list[i] == 1:
            plt.axvspan(i,i+1,facecolor='b', alpha=0.5)

    # Plot labels and axes setup
    plt.xlabel('Frame Number')
    plt.ylabel('Average Score over 20 evaluations')
#     blue_patch = mpatches.Patch(color='blue', alpha=0.5, label='Selected in Summary')
#     plt.legend(['Avg. Importance Score'], handles=[blue_patch])
    plt.legend(['Avg. Importance Score','Selected in Summary'], loc=1)
    # Create directories and save
    if not os.path.exists(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/"):
        os.makedirs(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/")
    print(f"{evaluation.proportion}_{evaluation.flexibility}/{file_id}")
    plt.savefig(f"../outputs/{evaluation.proportion}_{evaluation.flexibility}/{file_id}.png", dpi=300)
#     plt.show()
    plt.clf()

0.2_3/AwmHb44_ouw
0.2_3/98MoyGZKHXc
0.2_3/J0nA4VgnoCo
0.2_3/XzYM3PfTM4w
0.2_3/HT5vyqe0Xaw
0.2_3/vdmoEJ5YbrQ
0.2_3/xwqBXPGE9pQ
0.2_3/akI8YFjEmUw
0.2_3/i3wAGJaaktw
0.2_3/Bhxk-O1Y7Ho
0.2_3/0tmA_C6XwfM
0.2_3/3eYKfiOEJNs
0.2_3/WG0MBPpPC6I
0.2_3/Hl-__g2gn_A
0.2_3/Yi4Ij2NM7U4
0.2_3/37rzWOQsNIw
0.2_3/b626MiF1ew4
0.2_3/4wU_LUjG5Ic
0.2_3/WxtbjNsCQ8A
0.2_3/EE-bNr36nyA
0.2_3/Se3oxnaPsz0
0.2_3/qqR6AEXwxoQ
0.2_3/EYqVtI9YWJA
0.2_3/eQu1rNs0an0
0.2_3/JgHubY5Vw3Y
0.2_3/iVt07TCkFM0
0.2_3/E11zDS9XGzg
0.2_3/jcoYJXDG9sw



<Figure size 432x288 with 0 Axes>

In [91]:
clip_fps_data = np.array([np.array(i) for i in clip_fps])


In [93]:
from scipy.io import savemat
array = np.array([np.array(i, dtype=float) for i in all_val_list])
savemat('shots.mat', {"shot_boun": clip_fps_data})

In [131]:
# Generate graphs by varying the flexibility parameter.

clips = glob.glob('../outputs/varyf/*_clip_list*.json')
for i in clips:
    file_id = i.split('/')[-1].split('.')[0].split('_')[0]
    f = int(i.split('/')[-1].split('.')[0].split('_')[-1])
    print(video,f)
    frame_scores_for_video = df[df['id']==file_id].drop(axis=1, columns=['id','cat']).values
    frame_scores_for_video_nonan = frame_scores_for_video[~np.isnan(frame_scores_for_video)]
    frame_scores_for_video_nonan = frame_scores_for_video_nonan.reshape((20,-1))
    temp = []
    with open(f'../outputs/varyf/{file_id}_clip_list_{f}.json') as fil:
        clip_data = json.load(fil)

    # Convert clip start time and end time to fps
    fps = round(cv2.VideoCapture(f'../inputs/{file_id}.mp4').get(cv2.CAP_PROP_FPS))
    for clip in clip_data:
        h,m,s = clip['clip_start'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_start_fps'] = round(secs*fps)
        h,m,s = clip['clip_end'].split(':')
        secs = int(h)*3600+int(m)*60+float(s)
        clip['clip_end_fps'] = round(secs*fps)
        temp.append([clip['clip_start_fps'], clip['clip_end_fps']])
    # Generate list with included and excluded frames. Included frames are 1.
    val_list = [0 for i in range(frame_scores_for_video_nonan.shape[1])]
    try:
        for clip in clip_data:
            for i in range(clip['clip_start_fps'], clip['clip_end_fps']+1):
                val_list[i] = 1
        val_np = np.array(val_list)
    except:
        pass
    
#     for clip in clip_data:
#         print(clip['clip_start_fps'], clip['clip_end_fps'])

    # Plot the average score graph
    plt.plot(frame_scores_for_video_nonan.mean(axis=0))
    
    # Plot the regions of the selected frames
    for i in range(len(val_list)):
        if val_list[i] == 1:
            plt.axvspan(i,i+1,facecolor='b', alpha=0.5)

    # Plot labels and axes setup
    plt.xlabel('Frame Number')
    plt.ylabel('Average Score over 20 evaluations')
#     blue_patch = mpatches.Patch(color='blue', alpha=0.5, label='Selected in Summary')
#     plt.legend(['Avg. Importance Score'], handles=[blue_patch])
    plt.legend(['Avg. Importance Score','Selected in Summary'], loc=1)
    # Create directories and save
    plt.savefig(f"../outputs/varyf/{file_id}_{f}.png", dpi=300)
#     plt.show()
    plt.clf()
    

EE-bNr36nyA 1
EE-bNr36nyA 0
EE-bNr36nyA 7
EE-bNr36nyA 6
EE-bNr36nyA 5
EE-bNr36nyA 9
EE-bNr36nyA 8
EE-bNr36nyA 4
EE-bNr36nyA 10
EE-bNr36nyA 3
EE-bNr36nyA 2


<Figure size 432x288 with 0 Axes>